In [ ]:
#Sqs

In [20]:
import pylab as py
import math as ma
# from netCDF4 import Dataset

In [ ]:
# W updraft [m/s]
# Nd droplet size distribution with column names 'n' and 'r'
# 'n' is number concentration 'r' is radius 
# T is the temperature degreees celsius
# p is the pressure in hpa
# return to A two-componetn vextor containing the quasi-steady supersaturation
#example 
#Sqs (W=2., Nd=data.frame(r=2:20, n=c(rep(0,5), rep(50,10), rep(0,4))), T=4.3, p=771)

In [78]:
#' @title MurphyKoop
#' @description Returns the water vapor pressure corresponding to a supplied temperature.
#' @details Calculates the equilibrium water vapor pressure according to the 
#' Murphy-Koop equation
#' @param DP A numeric vector containing the dew point in deg. C
#' @param P An optional numeric vector representing the total pressure in hPa. Set zero 
#' to suppress the 'enhancement factor' correction for total pressure. Use the pressure to
#' find the equilibrium water vapor in the presence of that total pressure of air.
#' @return Water vapor pressure [hPa] in equilibrium with a plane water surface at dew point DP
#' @examples 
#' e <- MurphyKoop (-12.)
#' e <- MurphyKoop (10., 800.)
#' EW2 <- MurphyKoop (RAFdata$DPXC, RAFdata$PSXC)

def MurphyKoop(T, P=0.0):
    # returns vapor pressure via Murphy-Koop equations.
    # Supply DP=dewpoint (deg C) and optionally P=pressure (hPa),
    # the latter for the enhancement-factor correction.
    b0 = 9.550426
    b1 = -5723.265
    b2 = 3.53068
    b3 = -0.00728332
    b7 = 54.842763
    b8 = -6763.22
    b9 = -4.210
    b10 = 0.000367
    b11 = 0.0415
    TR2 = 218.8
    b12 = 53.878
    b13 = -1331.22
    b14 = -9.44523
    b15 = 0.014025
    tk = T + 273.15
    ess = ma.exp (b7 + b8 / tk + b9 * ma.log(tk) + b10 * tk + ma.tanh (b11 * (tk - TR2))
             * (b12 + b13 / tk + b14 * ma.log(tk) + b15 * tk))
    ess = ess/100.
    f = 1.e-5 * P * (4.923 - 0.0325 * tk + 5.84e-5 * tk**2)+1.
    ess = f * ess
    return(ess)

ess=MurphyKoop(4.3,771)
print(ess)

8.334658961794448


In [90]:
#first we calculat a1 and a2
#example T = 4.3, p = 771mb 

T = 4.3 #C
p = 771 #mb
W= 1 #m/s


Md= 28.9637 # molecular weight of dry air kg kmol-1
Mw = 18.0153 # molecular weight of dry water kg kmol-1
R0 = 8.314472e3 # J kmol-1 K-1 #universal gas constant 
Rd = R0/Md
Rw= R0/Mw
Cp = 1.00473e3 # specific heat of dry air at constant pressure J kg-1 K-1
Cv = 0.71766e3 # specific heat of constant volum J kg-1 K-1
lhv = (2.501 - 0.00237 * T) * 1.e6 # latent heat of vaporization J/kg
Tk = T+273.15
pi = 3.14
alpha = 1
beta = 0.04
rho = 100*p / (Rd*Tk)
r = 2 #um
es = MurphyKoop (T,771)

a1 = (9.80/Tk) * (lhv / (Cp*Rw*Tk) - 1/Rd)

#for a2, we need aa, ab, and r to get fa, fb.
C = 2.40e-2 * (Tk / 273.15)**0.91
D = 0.225 * 1.e-4 * (Tk / 273.15)**1.7 * 1013.25 / p
aa = C * ma.sqrt(2*pi*R0*Md*Tk) / (alpha * p * 100 * (3*R0))
ab = ma.sqrt(2*pi/(Rw*Tk)) * D / beta
fa =  r*1.e-6 / (r*1.e-6 + aa)
fb =  r*1.e-6 / (r*1.e-6 + ab)
a2 = ((4*pi/rho) * ((p*Rw/(Rd*es)) + lhv**2/(Rw*Cp*Tk**2)))/(Rw*Tk/(fb*D*es*100) + lhv**2/(fa*C*Rw*Tk**2))


a2I = a2*2*100
Sqs = a1*W/a2I #quasi-steady value of the supersaturation 
tau = 1/a2I #the time constant characterizing approach to Sqs



31.79642075345625